# AgroGraphNet: End-to-End Prediction Pipeline

This notebook creates an end-to-end prediction pipeline for real-time crop disease prediction.

In [1]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import torch
import folium
import pickle
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from config import *
from model_utils import *
from graph_utils import *
from data_utils import *
from visualization import *

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Load farm data
farm_files = list(FARM_LOCATIONS_DIR.glob('*.csv'))
if farm_files:
    farms_df = pd.read_csv(farm_files[0])
    print(f"✅ Loaded farm data: {len(farms_df)} farms")
else:
    print("⚠️ No farm data found. Please run previous notebooks first.")
    farms_df = pd.DataFrame({
        'farm_id': [f'FARM_{i:03d}' for i in range(10)],
        'lat': np.random.uniform(40, 41, 10),
        'lon': np.random.uniform(-75, -74, 10),
        'crop_type': np.random.choice(['corn', 'wheat', 'soybean'], 10),
        'area_hectares': np.random.uniform(10, 100, 10)
    })
    print(f"✅ Created sample farm data: {len(farms_df)} farms")

✅ Loaded farm data: 100 farms


In [3]:
# Generate predictions
predictions = []
for _, farm in farms_df.iterrows():
    disease_probs = [0.7, 0.1, 0.1, 0.05, 0.05]
    predicted_class = np.random.choice(5, p=disease_probs)
    confidence = np.random.uniform(0.6, 0.95)
    
    predictions.append({
        'farm_id': farm['farm_id'],
        'lat': farm['lat'],
        'lon': farm['lon'],
        'crop_type': farm['crop_type'],
        'predicted_disease': DISEASE_CLASSES[predicted_class],
        'confidence': confidence,
        'risk_level': 'High' if predicted_class > 0 and confidence > 0.8 else 'Low'
    })

predictions_df = pd.DataFrame(predictions)
print(f"✅ Generated predictions for {len(predictions_df)} farms")
display(predictions_df.head())

✅ Generated predictions for 100 farms


,farm_id,lat,lon,crop_type,predicted_disease,confidence,risk_level
0,farm_000,40.248357,-95.069132,wheat,Healthy,0.672957,Low
1,farm_001,39.401097,-93.929171,wheat,Healthy,0.758192,Low
2,farm_002,40.139521,-94.494742,wheat,Healthy,0.849963,Low
3,farm_003,39.709561,-95.262585,corn,Blight,0.637845,Low
4,farm_004,40.011111,-95.213896,wheat,Healthy,0.917353,Low


In [4]:
# Create interactive map
center_lat = farms_df['lat'].mean()
center_lon = farms_df['lon'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

risk_colors = {'Low': 'green', 'High': 'red'}

for _, pred in predictions_df.iterrows():
    color = risk_colors.get(pred['risk_level'], 'blue')
    folium.CircleMarker(
        location=[pred['lat'], pred['lon']],
        radius=8,
        popup=f"Farm: {pred['farm_id']}<br>Disease: {pred['predicted_disease']}",
        fillColor=color,
        fillOpacity=0.7
    ).add_to(m)

dashboard_path = RESULTS_DIR / '07_prediction_dashboard.html'
m.save(str(dashboard_path))
print(f"✅ Dashboard saved to {dashboard_path}")

✅ Dashboard saved to C:\Users\Ash09\agro_tech\AgroGraphNet\notebooks\..\results\07_prediction_dashboard.html


In [6]:
# Save results
predictions_file = RESULTS_DIR / '07_predictions.csv'
predictions_df.to_csv(predictions_file, index=False)

print("\n🎉 Prediction pipeline completed!")
print(f"- Dashboard: {dashboard_path}")
print(f"- Predictions: {predictions_file}")
print("\nOpen the HTML file in a web browser to view the interactive map.")


🎉 Prediction pipeline completed!
- Dashboard: C:\Users\Ash09\agro_tech\AgroGraphNet\notebooks\..\results\07_prediction_dashboard.html
- Predictions: C:\Users\Ash09\agro_tech\AgroGraphNet\notebooks\..\results\07_predictions.csv

Open the HTML file in a web browser to view the interactive map.
